In [70]:
import pandas as pd
import os

In [71]:
# 엑셀 파일 경로
excel_path = "raw_data/25_2_department_of_economics.xlsx"

# 엑셀 로드 (첫 시트 기준)
excel_df = pd.read_excel(excel_path, engine="openpyxl")

# DataFrame 확인
excel_df.head()

,대학,개설학과,학년,과정,이수구분,과목번호-분반,과목명,학점-시간,담당교수,폐강,강의시간,유연학기,비고,수업유형
0,경영경제대학,경제학부(서울),1,학사,자유선택,51097-01,CAU세미나(2),.5-.5,이종철,NaN,화10 / 310관 927호 <강의실>,NaN,NaN,NaN
1,경영경제대학,경제학부(서울),1,학사,자유선택,51097-02,CAU세미나(2),.5-.5,이정희,NaN,월10 / 310관 932호 <강의실>,NaN,NaN,NaN
2,경영경제대학,경제학부(서울),1,학사,자유선택,51097-03,CAU세미나(2),.5-.5,이한영,NaN,목10 / 310관 927호 <강의실>,NaN,NaN,NaN
3,경영경제대학,경제학부(서울),1,학사,자유선택,51097-04,CAU세미나(2),.5-.5,진현정,NaN,월10 / 310관 804호 <강의실>,NaN,NaN,NaN
4,경영경제대학,경제학부(서울),1,학사,자유선택,51097-05,CAU세미나(2),.5-.5,강창희,NaN,화10 / 310관 802호 <강의실>,NaN,NaN,NaN


In [72]:
# 1. 과목번호, 과목명, 담당교수, 폐강, 강의시간만 남기고 제거
columns_to_keep = ["과목번호-분반", "과목명", "담당교수", "폐강", "강의시간"]
excel_df = excel_df[columns_to_keep]
excel_df.head()

,과목번호-분반,과목명,담당교수,폐강,강의시간
0,51097-01,CAU세미나(2),이종철,NaN,화10 / 310관 927호 <강의실>
1,51097-02,CAU세미나(2),이정희,NaN,월10 / 310관 932호 <강의실>
2,51097-03,CAU세미나(2),이한영,NaN,목10 / 310관 927호 <강의실>
3,51097-04,CAU세미나(2),진현정,NaN,월10 / 310관 804호 <강의실>
4,51097-05,CAU세미나(2),강창희,NaN,화10 / 310관 802호 <강의실>


In [73]:
# 1. 폐강 열이 NaN이 아닌 행 제거
excel_df = excel_df[excel_df["폐강"].isna()]

# 2. 폐강 열 제거
excel_df = excel_df.drop(columns=["폐강"])
excel_df.head()

,과목번호-분반,과목명,담당교수,강의시간
0,51097-01,CAU세미나(2),이종철,화10 / 310관 927호 <강의실>
1,51097-02,CAU세미나(2),이정희,월10 / 310관 932호 <강의실>
2,51097-03,CAU세미나(2),이한영,목10 / 310관 927호 <강의실>
3,51097-04,CAU세미나(2),진현정,월10 / 310관 804호 <강의실>
4,51097-05,CAU세미나(2),강창희,화10 / 310관 802호 <강의실>


In [74]:
def convert_period_to_time(period_num: int) -> str:
    # 0교시: 8시 시작, 그 이후는 9시부터 시작
    base_hour = 8 if period_num == 0 else 9 + (period_num - 1)
    start = f"{base_hour:02d}:00"
    end = f"{base_hour + 1:02d}:00"
    return f"{start}~{end}"

import re

def parse_schedule(schedule_str: str):
    if pd.isna(schedule_str):
        return None, None, None

    # 시간 정보 추출

    # 예: '월10 / 310관 802호' 또는 '월(10:30~11:45) / 수10 / 303관 802호'
    parts = schedule_str.split('/')
    time_parts = []
    room_part = None

    for part in parts:
        part = part.strip()

        # 교시 형식 예: 월0, 수3 등
        match_period = re.match(r"([월화수목금토일])(\d+)", part)
        if match_period:
            day = match_period.group(1)
            period = int(match_period.group(2))
            time_range = convert_period_to_time(period)
            time_parts.append(f"{day}({time_range})")
            continue

        # 이미 시간으로 되어 있으면 그대로 추가
        match_time = re.match(r"([월화수목금토일])\(.+\)", part)
        if match_time:
            time_parts.append(part)
            continue

        # 강의실 정보로 간주
        if re.search(r"\d+관.*\d+호", part):
            room_part = part

    # 건물번호, 강의실 분리
    building = None
    classroom = None
    if room_part:
        match_room = re.match(r"(\d+관).*?(\d+-?\d*호)", room_part)
        if match_room:
            building = match_room.group(1)
            classroom = match_room.group(2)

    return " / ".join(time_parts), building, classroom

def split_time_info(time_str):
    if pd.isna(time_str):
        return None, None

    days = []
    times = []

    # 예: 월(10:30~11:45)
    parts = [p.strip() for p in time_str.split('/')]

    for part in parts:
        match = re.match(r"([월화수목금토일])\((\d{2}:\d{2}~\d{2}:\d{2})\)", part)
        if match:
            days.append(match.group(1))
            times.append(match.group(2))

    return days, times


# 강의시간, 건물번호, 강의실 파싱해서 새 컬럼으로 추가
excel_df[["강의시간_24시", "건물번호", "강의실"]] = excel_df["강의시간"].apply(
    lambda x: pd.Series(parse_schedule(x))
)

excel_df[["강의요일", "강의시간_리스트"]] = excel_df["강의시간_24시"].apply(
    lambda x: pd.Series(split_time_info(x))
)

excel_df = excel_df.drop(columns=["강의시간", "강의시간_24시"])

# 확인
excel_df.head()

,과목번호-분반,과목명,담당교수,건물번호,강의실,강의요일,강의시간_리스트
0,51097-01,CAU세미나(2),이종철,310관,927호,[화],[18:00~19:00]
1,51097-02,CAU세미나(2),이정희,310관,932호,[월],[18:00~19:00]
2,51097-03,CAU세미나(2),이한영,310관,927호,[목],[18:00~19:00]
3,51097-04,CAU세미나(2),진현정,310관,804호,[월],[18:00~19:00]
4,51097-05,CAU세미나(2),강창희,310관,802호,[화],[18:00~19:00]


In [75]:
excel_df.iloc[37:43]

,과목번호-분반,과목명,담당교수,건물번호,강의실,강의요일,강의시간_리스트
37,50023-67,COMMUNICATION IN ENGLISH,이안고든,203관,409호,"[화, 목]","[15:00~16:15, 15:00~16:15]"
38,01415-01,경제수학,송정석,310관,927호,"[화, 목]","[16:30~17:45, 16:30~17:45]"
39,01415-04,경제수학,NaN,310관,513호,[금],[09:00~10:00]
40,01415-02,경제수학 (영어A강의),박성용,310관,925호,"[화, 목]","[09:00~10:15, 09:00~10:15]"
41,01415-03,경제수학 (영어A강의),심재훈,310관,513호,"[화, 목]","[15:00~16:15, 15:00~16:15]"
42,01498-01,경제학개론(1),강상구,310관,412호,"[월, 수]","[15:00~16:15, 15:00~16:15]"


In [76]:
# zip 요일과 시간 리스트 → 튜플 목록
excel_df["temp"] = excel_df.apply(
    lambda row: list(zip(row["강의요일"], row["강의시간_리스트"])), axis=1
)

# explode
df_exploded = excel_df.explode("temp")

# 튜플만 남기기
df_exploded = df_exploded[df_exploded["temp"].apply(lambda x: isinstance(x, tuple) and len(x) == 2)]

# 튜플 분리
df_exploded[["요일", "시간"]] = pd.DataFrame(df_exploded["temp"].tolist(), index=df_exploded.index)
df_exploded = df_exploded.drop(columns=["temp", "강의요일", "강의시간_리스트"])

In [77]:
df_exploded[["시작시간", "종료시간"]] = df_exploded["시간"].str.split("~", expand=True)
df_exploded = df_exploded.drop(columns=["시간"])

In [78]:
df_exploded = df_exploded.rename(columns={
    "건물번호": "building",
    "강의실": "room",
    "요일": "day",
    "시작시간": "start_time",
    "종료시간": "end_time",
    "과목번호-분반": "course_id",
    "과목명": "course_name",
    "담당교수": "professor"
})

In [79]:
df_exploded = df_exploded[
    ["building", "room", "day", "start_time", "end_time", "course_id", "course_name", "professor"]
]

In [80]:
# building에서 숫자만 추출 (예: "310관" → "310")
df_exploded["building"] = df_exploded["building"].apply(
    lambda x: re.search(r"\d+", str(x)).group() if pd.notna(x) else None
)

# room에서도 숫자만 추출 (예: "415호" 또는 "802-1호" → "415", "802-1")
df_exploded["room"] = df_exploded["room"].apply(
    lambda x: re.search(r"\d+(?:-\d+)?", str(x)).group() if pd.notna(x) else None
)

In [81]:
day_map = {
    "월": "monday",
    "화": "tuesday",
    "수": "wednesday",
    "목": "thursday",
    "금": "friday",
    "토": "saturday",
    "일": "sunday"
}

df_exploded["day"] = df_exploded["day"].map(day_map)

In [82]:
df_exploded = df_exploded.dropna(subset=["building", "room"])

In [83]:
df_exploded["building"] = df_exploded["building"].astype(int)


In [84]:
df_exploded.tail(20)

,building,room,day,start_time,end_time,course_id,course_name,professor
82,310,802,wednesday,10:30,11:45,56107-01,동태적 경제이론 (영어A강의),문지웅
83,310,921,friday,15:00,16:00,10235-02,산업조직론,김형진
86,310,612,monday,15:00,16:15,54461-02,응용계량경제학,NaN
86,310,612,wednesday,15:00,16:15,54461-02,응용계량경제학,NaN
87,310,927,monday,13:30,14:45,24658-01,화폐금융론,송원호
87,310,927,wednesday,13:30,14:45,24658-01,화폐금융론,송원호
88,310,311,tuesday,13:30,14:45,24658-02,화폐금융론,김태진
88,310,311,thursday,13:30,14:45,24658-02,화폐금융론,김태진
89,310,505,thursday,12:00,13:00,24658-03,화폐금융론 (영어A강의),구세범
90,310,921,monday,09:00,10:15,01452-01,경제정책론 (영어A강의),고선


In [85]:
json_data = df_exploded.to_dict(orient="records")

save_dir = "converted_data"
os.makedirs(save_dir, exist_ok=True)

import json

save_path = os.path.join(save_dir, "room_schedule.json")

with open(save_path, "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=2)

print(f"✅ JSON 저장 완료: {save_path}")


✅ JSON 저장 완료: converted_data\room_schedule.json
